In [9]:
import numpy as np
import cv2
import time
import pysuperansac
import random
import cv2
import pycolmap
import poselib
from scipy.spatial.transform import Rotation as R
import pandas as pd

In [10]:
# Loading example data
correspondences = np.loadtxt('data/pose6dscene_points.txt')
gt_pose = np.loadtxt('data/pose6dscene_gt.txt')
intrinsics = np.loadtxt('data/pose6dscene.K')

print("Number of correspondences loaded = ", str(len(correspondences)))

repetition_number = 10

Number of correspondences loaded =  95


In [11]:
def run_opencv(normalized_corrs, K, threshold):
    n = len(normalized_corrs)
    imagePoints = np.float32([normalized_corrs[i][0:2] for i in np.arange(n)]).reshape(-1,2)
    worldPoints = np.float32([normalized_corrs[i][2:5] for i in np.arange(n)]).reshape(-1,3)
    dist_coeffs = np.zeros((4,1))
    camera_matrix = np.identity(3)

    normalized_threshold = threshold / (K[0, 0] + K[1, 1]) / 2.0;    

    success, rotation_vector, translation_vector, inliers = cv2.solvePnPRansac(
        worldPoints, 
        imagePoints, 
        camera_matrix, 
        dist_coeffs, 
        flags = cv2.SOLVEPNP_ITERATIVE,
        iterationsCount = 1000,
        reprojectionError = normalized_threshold)
    
    mask = np.zeros(n)
    if not success:
        return np.zeros((3, 4)), mask
    mask[inliers] = 1
    
    rotation, _ = cv2.Rodrigues(rotation_vector)
    pose = np.concatenate((rotation, translation_vector), axis=1)
    return pose, mask            

def run_superansac(matches, K, config):
    cfg = {
        "model": "SIMPLE_PINHOLE",
        "width": int(K[0,2] * 2),
        "height": int(K[1,2] * 2),
        "params": [K[0,0], K[0,2], K[1,2]],
    }

    query_camera = pycolmap.Camera(cfg)
    
    # Run the fundamental matrix estimation implemented in SupeRANSAC
    tic = time.perf_counter()
    rotation, translation, inliers, score, iterations = pysuperansac.estimateAbsolutePose(
        np.ascontiguousarray(matches), 
        pysuperansac.CameraType.SimplePinhole,
        query_camera.params,
        [],
        config = config)
    toc = time.perf_counter()
    elapsed_time = toc - tic

    mask = np.zeros((matches.shape[0], 1), dtype=np.uint8)
    mask[inliers] = 1

    return rotation, translation, mask       

def run_poselib(matches, K, threshold):
    camera = { 'model': "SIMPLE_RADIAL", 'width': int(K[0,2] * 2), 'height': int(K[1,2] * 2), 'params': [K[0,0], K[0,2], K[1,2]] }
        
    points2D = np.array(matches[:,:2]).astype(np.float64)
    points3D = np.array(matches[:,2:]).astype(np.float64)

    pose, info = poselib.estimate_absolute_pose(
        points2D, 
        points3D, 
        camera, 
        { 'max_reproj_error': threshold }, 
        {})
    
    # Convert to w,x,y,z
    qvec = pose.q
    qvec = np.array([qvec[1], qvec[2], qvec[3], qvec[0]])
    rotation = R.from_quat(qvec).as_matrix()
    translation = pose.t
    
    return rotation, translation, info['inliers']

def normalize_image_points(corrs, K): 
    n = len(corrs)
    normalized_correspondences = np.zeros((corrs.shape[0], 5))
    inv_K = np.linalg.inv(K)

    for i in range(n):
        p1 = np.append(corrs[i][0:2], 1)
        p2 = inv_K.dot(p1)
        normalized_correspondences[i][0:2] = p2[0:2]
        normalized_correspondences[i][2:] = corrs[i][2:]
    return normalized_correspondences

def calculate_error(gt_pose, est_pose):
    R2R1 = np.dot(gt_pose[:, 0:3].T, est_pose[:, 0:3])
    cos_angle = max(-1.0, min(1.0, 0.5 * (R2R1.trace() - 1.0)))
    
    err_R = np.arccos(cos_angle) * 180.0 / np.pi
    err_t = np.linalg.norm(gt_pose[:, 3] - est_pose[:, 3])
    
    return err_R, err_t

In [12]:

results_superansac = []
results_poselib = []
results_cv2 = []

threshold = 3.0

for noise in [0.0, 0.5, 1.0, 3.0, 5.0]:
    for outlier_ratio in [0.0, 1.0, 2.0, 5.0, 10.0]:
        # Add outliers to the correspondences
        outlier_number = round(outlier_ratio * correspondences.shape[0])
        mins = np.min(correspondences, axis=0)
        maxs = np.max(correspondences, axis=0)
        
        outliers = []
        
        outliers = np.zeros((outlier_number,5))
        for dim in range(5):
            outliers[:,dim] = np.random.uniform(mins[dim], maxs[dim], outliers[:,dim].shape)
        new_correspondences = np.concatenate((correspondences, outliers), axis=0)
        new_correspondences[:,:2] += np.random.normal(0, noise, new_correspondences[:,:2].shape)
        
        # Normalize the points
        normalized_correspondences = normalize_image_points(new_correspondences, intrinsics)

        for rep in range(repetition_number):
            # Set up the configuration
            config = pysuperansac.RANSACSettings()
            config.inlier_threshold = threshold
            config.min_iterations = 1000
            config.max_iterations = 10000
            config.confidence = 0.999
            config.sampler = pysuperansac.SamplerType.PROSAC
            config.scoring = pysuperansac.ScoringType.MAGSAC
            config.local_optimization = pysuperansac.LocalOptimizationType.NestedRANSAC
            config.final_optimization = pysuperansac.LocalOptimizationType.IteratedLSQ

            # Run OpenCV RANSAC 
            pose, mask = run_opencv(normalized_correspondences, intrinsics, threshold = threshold)
            err_R, err_t = calculate_error(gt_pose, pose)
            results_cv2.append([err_R, err_t, np.sum(mask)])

            # Run PoseLib
            rot, t, mask = run_poselib(new_correspondences, intrinsics, threshold = threshold)
            pose = np.concatenate((rot, np.resize(t, (3, 1))), axis=1)
            err_R, err_t = calculate_error(gt_pose, pose)
            results_poselib.append([err_R, err_t, np.sum(mask)])

            # Run SupeRANSAC 
            rot, t, mask = run_superansac(new_correspondences, intrinsics, config)
            pose = np.concatenate((rot, np.resize(t, (3, 1))), axis=1)
            err_R, err_t = calculate_error(gt_pose, pose)
            results_superansac.append([err_R, err_t, np.sum(mask)])

avg_results_poselib = np.array(results_poselib).mean(axis=0)
avg_results_cv2 = np.array(results_cv2).mean(axis=0)
avg_results_superansac = np.array(results_superansac).mean(axis=0)

# Create the DataFrame
df = pd.DataFrame({
    "Metric": ["Rotation error (°)", "Translation error (cm)", "Inlier number"],
    "cv2": [f"{avg_results_cv2[0]:.3f}", f"{avg_results_cv2[1]:.3f}", f"{avg_results_cv2[2]:.0f}"],
    "PoseLib": [f"{avg_results_poselib[0]:.3f}", f"{avg_results_poselib[1]:.3f}", f"{avg_results_poselib[2]:.0f}"],
    "SupeRANSAC": [f"{avg_results_superansac[0]:.3f}", f"{avg_results_superansac[1]:.3f}", f"{avg_results_superansac[2]:.0f}"]
})

df

,Metric,cv2,PoseLib,SupeRANSAC
0,Rotation error (°),52.573,12.057,10.013
1,Translation error (cm),531.019,55.593,49.765
2,Inlier number,24,74,73
